In [4]:
from io import TextIOWrapper
import re


def line_batches(input: TextIOWrapper):
    lines = ["", *(input.readline() for _ in range(0, 2))]
    yield lines
    while new_line := input.readline():
        lines = [*lines[1:], new_line]
        yield lines
    lines = [*lines[1:], ""]
    yield lines

def circle_around(x: tuple[int, int], y: int):
    def sign(num: int):
        return 1 if num > 0 else -1 if num < 0 else 0

    startX, endX = x

    path: list[tuple[int, int]] = [
        (startX - 1, y - 1),
        (endX + 1, y - 1),
        (endX + 1, y + 1),
        (startX - 1, y + 1),
        (startX - 1, y - 1),
    ]

    pos = path[0]
    for dest in path[1:]:
        x1, y1 = pos
        x2, y2 = dest
        dir = (sign(x2 - x1), sign(y2 - y1))
        while pos != dest:
            yield pos
            pos = (pos[0] + dir[0], pos[1] + dir[1])

def adjacent_symbol(line_batch: list[str], num_match: re.Match[str]):
    start, end = num_match.span()
    for x, y in circle_around((start, end - 1), 1):
        line = line_batch[y]
        if not line:
            continue
        if sym_match := re.fullmatch(r"[^0-9\.\s]", line[x]):
            return sym_match
    return None

with open("./input.txt") as input:
    num_sum = 0
    for kernel in line_batches(input):
        line = kernel[1]
        for num_match in re.finditer(r"(\d+)", line):
            if sym_match := adjacent_symbol(kernel, num_match):
                num_sum += int(num_match.group())
    print(num_sum)

539590


In [35]:
from math import prod
import re
from typing import Optional, Union


def adjacent_symbols(line_batch: list[str], num_match: re.Match[str]):
    syms: list[tuple[re.Match[str], tuple[int, int]]] = []

    start, end = num_match.span()
    for x, y in circle_around((start, end - 1), 1):
        line = line_batch[y]
        if not line:
            continue
        if sym_match := re.fullmatch(r"([^0-9\.\s])", line[x]):
            syms.append((sym_match, (x, y)))
    return syms


with open("./input.txt") as input:
    line_num = -1
    star_nums: dict[tuple[int, int], list[int]] = {}
    gear_sum = 0

    def flush_star_nums(line_num: Optional[int] = None):
        sum = 0
        rm_keys: list[tuple[int, int]] = []
        for key in star_nums.keys():
            x, y = key
            if line_num is None or y < line_num:
                if len(star_nums[key]) == 2:
                    sum += prod(star_nums[key])
                rm_keys.append(key)
        for rm_key in rm_keys:
            star_nums.pop(rm_key)
        return sum


    for kernel in line_batches(input):
        line_num += 1
        line = kernel[1]

        gear_sum += flush_star_nums(line_num -1)

        # Link stars to adjacent nums
        for num_match in re.finditer(r"(\d+)", line):
            for sym, pos in adjacent_symbols(kernel, num_match):
                if sym.group() == "*":
                    global_pos = (pos[0], pos[1] + line_num - 1)
                    nums = star_nums.get(global_pos, [])
                    nums.append(int(num_match.group()))
                    star_nums[global_pos] = nums

    flush_star_nums()
    assert len(star_nums) == 0
    print(gear_sum)
    # 80703636

80703636
